In [1]:
import numpy as np
import matplotlib.pylab as plt

In [2]:
N_agents = 2
N_actions = 3
matrix_shape = (2,3,4,5,6)
print(matrix_shape)
payoff_matrix_shape = tuple([N_actions for i in range(N_agents)] + [N_agents])
payoff_matrix = np.reshape(np.array([ 3,  8,  4,  0,  5,  3,  1, -3,  9,  4,  8,  5,  4,  7,  2,  6,  2, 9]),(3,3,2))
payoff_matrix = payoff_matrix.transpose(1, 0, 2)
def relu(x):
    return np.maximum(0, x)
# set payoff_matrix:
matrix = np.random.normal(1, 4.5, size=matrix_shape)

(2, 3, 4, 5, 6)


In [3]:
matrix[(1,2,slice(None),4,5)]

array([-2.8554923 ,  3.31293776,  2.71467303, -1.77834912])

In [4]:
payoff_matrix = np.random.normal(1, 4.5, size=payoff_matrix_shape)

In [5]:
def normalize_per_row(matrix):
    N, M = matrix.shape
    matrix[:] = relu(matrix)
    for i in range(N):
        total = np.sum(matrix[i])
        if total== 0:
            matirx[i][0] = 1
        else:
            matrix[i] = matrix[i]/total

In [6]:
global strategies_matrix 
strategies_matrix = np.ones((N_agents, N_actions))/N_actions

In [7]:
def project_grad(g):
  return g - g.sum() / g.size

In [8]:
histories_matrix = np.zeros_like(strategies_matrix)

grad_histories =  np.zeros_like(histories_matrix)

grad_strategies = np.zeros_like(strategies_matrix)

other_player_fx = np.zeros_like(strategies_matrix)

reg_exp = np.zeros(N_agents)

exp_thresh = 0.01

lr_strategy = 0.001

lr_history = 0.02

anneal_steps = 0


from scipy import special

In [15]:
# take an agent's action: np.random.choice(N_actions, p=strategies_matrix[agent])


temp = 100
def single_iter(t,strategies_matrix,histories_matrix,grad_histories,grad_strategies,other_player_fx,reg_exp,temp,anneal_steps,lr_history,lr_strategy,exp_thresh):
    if t%1000 == 0:
        print(t)
    actions = np.array([np.random.choice(N_actions, p=strategies_matrix[agent]) for agent in range(N_agents)])
    for i in range(N_agents):
        nabla_i = np.zeros_like(strategies_matrix[i])
        for j in range(N_agents):
            if i == j:
                continue
                
            actions_with_holes = list(actions.copy()) + [i]
            actions_with_holes[i] = slice(None)
            actions_with_holes[j] = slice(None)
            actions_with_holes = tuple(actions_with_holes)
            hess_i_ij = payoff_matrix[actions_with_holes]
            nabla_ij = hess_i_ij.dot(strategies_matrix[i])
            nabla_i += nabla_ij/ float(N_agents - 1)
        grad_histories[i][:] = histories_matrix[i][:] - nabla_i
        
        br_i = special.softmax(histories_matrix[i] / temp)
        br_i_mat = (np.diag(br_i) - np.outer(br_i, br_i)) / temp
        br_i_strategies_gradient = nabla_i - temp * (np.log(br_i) + 1)

        strategies_gradient_i = nabla_i - temp * (np.log(strategies_matrix[i] + 0.0001) + 1)

        other_player_fx[i][:] = (br_i - strategies_matrix[i]) + br_i_mat.dot(br_i_strategies_gradient)

        entr_br_i = temp * special.entr(br_i).sum()
        entr_strategies_i = temp* special.entr(strategies_matrix[i]).sum()

        reg_exp[i] = (histories_matrix[i].dot(br_i - strategies_matrix[i]) + entr_br_i - entr_strategies_i)

        grad_strategies[i][:] -= strategies_gradient_i
        for j in range(N_agents):
            if i == j:
                continue
                
            actions_with_holes = list(actions.copy()) + [j]
            actions_with_holes[i] = slice(None)
            actions_with_holes[j] = slice(None)
            actions_with_holes = tuple(actions_with_holes)
            hess_j_ij = payoff_matrix[actions_with_holes]
            grad_strategies [i][:]  += hess_j_ij.dot(other_player_fx[j])
            
        grad_strategies[i][:] = project_grad(grad_strategies[i][:])
        if np.isnan(grad_strategies).any():
            print("nanananan!")
    reg_exp_mean = np.mean(reg_exp)
    if (reg_exp_mean < exp_thresh) and (anneal_steps >= 1/lr_history):
        temp = np.clip(temp/ 2., 0., 1.)
        d_anneal_steps = -anneal_steps
    else:
        d_anneal_steps = 1

    #update
    lr_h = np.clip(1 / float(t + 1), lr_history, np.inf)
    anneal_steps += d_anneal_steps 
    histories_matrix -= lr_h*grad_histories
    strategies_matrix[:] = np.clip(strategies_matrix, 0.000001, np.inf) - lr_strategy * grad_strategies
    normalize_per_row(strategies_matrix)
    


In [16]:
for i in range(100000, 200000):
    single_iter(i,strategies_matrix,histories_matrix,grad_histories,grad_strategies,other_player_fx,reg_exp,temp,anneal_steps,lr_history,lr_strategy,exp_thresh)

print(strategies_matrix)

100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
156000
157000
158000
159000
160000
161000
162000
163000
164000
165000
166000
167000
168000
169000
170000
171000
172000
173000
174000
175000
176000
177000
178000
179000
180000
181000
182000
183000
184000
185000
186000
187000
188000
189000
190000
191000
192000
193000
194000
195000
196000
197000
198000
199000
[[0.25902862 0.         0.74097138]
 [0.25329985 0.55914656 0.18755358]]
